# Language translator: English to French

Vivek Viswam R. V. <br>
Rachel Messenger

### Imports

In [ ]:
import os
import warnings

import torch
import torch.nn as nn
import torch.optim as optim
import spacy

from torch.utils.data import Dataset, DataLoader
from collections import Counter

!python -m pip install torchmetrics
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm

from torchmetrics.text import BLEUScore

warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 25.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Constants

In [ ]:
ROOT_PATH = "/content/drive/MyDrive/language-translator"
EN_REL_PATH = "dataset/europarl-v7.fr-en.en"
FR_REL_PATH = "dataset/europarl-v7.fr-en.fr"
TRAIN_STATS_FILENAME = "training_stats.pth"

EN_PATH = os.path.join(ROOT_PATH, EN_REL_PATH)
FR_PATH = os.path.join(ROOT_PATH, FR_REL_PATH)
TRAIN_STATS_PATH = os.path.join(ROOT_PATH, TRAIN_STATS_FILENAME)

START_TOKEN = "<START>"
END_TOKEN = "<END>"
PAD_TOKEN = "<PAD>"
UNKN_TOKEN = "<UNKN>"

MAX_SENT_LEN = 100
MAX_SEQ_LEN = MAX_SENT_LEN + 2
MAX_LINE_COUNT = 200000

### Custom dataset class definitions

In [ ]:
class LangDataset(Dataset):
  START_TOKEN = START_TOKEN
  END_TOKEN = END_TOKEN
  PAD_TOKEN = PAD_TOKEN
  UNKN_TOKEN = UNKN_TOKEN
  SPECIAL_TOKENS = ( START_TOKEN, END_TOKEN, PAD_TOKEN, UNKN_TOKEN )

  def __init__(self, src_path, tgt_path, src_tokenizer, tgt_tokenizer, min_tok_freq=2, max_lines=None, max_sent_len=MAX_SENT_LEN):
    self.src = None
    self.tgt = None

    self.src_path = src_path
    self.tgt_path = tgt_path
    self.max_sent_len = max_sent_len
    self.max_seq_len = max_sent_len + 2
    self.min_tok_freq = min_tok_freq

    self.src_lines = []
    self.tgt_lines = []
    self.src_idx2tok = {}
    self.src_tok2idx = {}
    self.tgt_idx2tok = {}
    self.tgt_tok2idx = {}
    self.src_vocab_size = 0
    self.tgt_vocab_size = 0

    self._src_tokenizer = src_tokenizer
    self._tgt_tokenizer = tgt_tokenizer
    self._max_lines = max_lines


    self._load_dataset()
    self._preprocess_dataset()
    self._create_vocab()
    self._tensorify_dataset()

  def tokenize_sentence(self, sentence, lang_type="src"):
      tokenize = self._src_tokenizer if lang_type == "src" else self._tgt_tokenizer
      tok2idx = self.src_tok2idx if lang_type == "src" else self.tgt_tok2idx
      counter = self.src_tok_ctr if lang_type == "src" else self.tgt_tok_ctr

      tokens = tokenize(sentence)
      tokens = [ self.UNKN_TOKEN if counter[tok] < self.min_tok_freq else tok for tok in tokens ]
      tokens = [ self.START_TOKEN ] + tokens + [ self.END_TOKEN ]
      padding_size = self.max_seq_len - len(tokens)
      tokens += [ self.PAD_TOKEN ] * padding_size
      tokens = [ tok2idx[tok] for tok in tokens ]

      return tokens

  def detokenize_tensor(self, tensor, lang_type="src"):
      idx2tok = self.src_idx2tok if lang_type == "src" else self.tgt_idx2tok
      words = [ idx2tok[idx.item()] for idx in tensor ]

      return words

  def _load_dataset(self):
    print("Reading source and target files...")

    with open(self.src_path) as src_f, open(self.tgt_path) as tgt_path:
      src_lines = src_f.read().splitlines()
      tgt_lines = tgt_path.read().splitlines()

      self.src_lines = src_lines[:self._max_lines] if self._max_lines else src_lines
      self.tgt_lines = tgt_lines[:self._max_lines] if self._max_lines else tgt_lines

    print(f"Reading source files! Read {len(src_lines)} lines. Keeping {len(self.src_lines)} lines.")

  def _preprocess_dataset(self):
    print("Preprocessing dataset...")

    final_src_lines = []
    final_tgt_lines = []
    total_lines = len(self.src_lines)

    self.src_tok_ctr = Counter()
    self.tgt_tok_ctr = Counter()

    self._src_tok_set = set()
    self._tgt_tok_set = set()

    for idx in range(total_lines):
      src_line = self.src_lines[idx].lower()
      tgt_line = self.tgt_lines[idx].lower()

      src_line_toks = self._src_tokenizer(src_line)
      tgt_line_toks = self._tgt_tokenizer(tgt_line)

      src_line_len = len(src_line_toks)
      tgt_line_len = len(tgt_line_toks)

      if src_line_len <= self.max_sent_len and tgt_line_len <= self.max_sent_len:
        final_src_lines.append(src_line)
        final_tgt_lines.append(tgt_line)

        self._src_tok_set |= set(src_line_toks)
        self._tgt_tok_set |= set(tgt_line_toks)

        self.src_tok_ctr.update(src_line_toks)
        self.tgt_tok_ctr.update(tgt_line_toks)

    self.src_lines, self.tgt_lines = final_src_lines, final_tgt_lines

    print(f"Preprocessing completed! Dropped {total_lines - len(self.src_lines)}/{total_lines} lines!")

  def _create_vocab(self):
    print("Creating vocabularies...")

    src_tokens = list(self.SPECIAL_TOKENS) + list(self._src_tok_set)
    tgt_tokens = list(self.SPECIAL_TOKENS) + list(self._tgt_tok_set)

    del self._src_tok_set
    del self._tgt_tok_set

    for idx, tok in enumerate(src_tokens):
      self.src_tok2idx[tok] = idx
      self.src_idx2tok[idx] = tok

    for idx, tok in enumerate(tgt_tokens):
      self.tgt_tok2idx[tok] = idx
      self.tgt_idx2tok[idx] = tok

    self.src_vocab_size = len(self.src_tok2idx)
    self.tgt_vocab_size = len(self.tgt_tok2idx)

    print(f"Created vocabularies! Source vocab size: {self.src_vocab_size} and target vocab size: {self.tgt_vocab_size}")

  def _tensorify_dataset(self):
    print("Tensorifying dataset...")

    final_src_lines = [ self.tokenize_sentence(line, "src") for line in self.src_lines ]
    final_tgt_lines = [ self.tokenize_sentence(line, "tgt") for line in self.tgt_lines ]

    self.src = torch.tensor(final_src_lines)
    self.tgt = torch.tensor(final_tgt_lines)

    print("Dataset tensorified!")

  def __len__(self):
    return len(self.src)

  def __getitem__(self, idx):
    return self.src[idx], self.tgt[idx]

### Class definitions

In [ ]:
class Transformer(nn.Module):
  def __init__(self,
               embedding_size,
               num_heads,
               num_encoder_layers,
               num_decoder_layers,
               dim_feedforward,
               dropout,
               src_voc_size,
               tgt_voc_size,
               src_pad_idx,
               tgt_pad_idx,
               device,
               max_sent_len=MAX_SENT_LEN,
    ):

    super(Transformer, self).__init__()

    self.device = device
    self.src_pad_idx = src_pad_idx
    self.tgt_pad_idx = tgt_pad_idx
    self.max_seq_len = max_sent_len + 2

    self.dropout = nn.Dropout(dropout)
    self.src_word_embedding = nn.Embedding(src_voc_size, embedding_size)
    self.src_position_embedding = nn.Embedding(self.max_seq_len, embedding_size)
    self.tgt_word_embedding = nn.Embedding(tgt_voc_size, embedding_size)
    self.tgt_position_embedding = nn.Embedding(self.max_seq_len, embedding_size)
    self.transformer = nn.Transformer(
        embedding_size,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        dim_feedforward,
        dropout
    )

    self.fc_out = nn.Linear(embedding_size, tgt_voc_size)

  def create_padding_mask(self, seq, pad_idx):
    return seq.transpose(0, 1) == pad_idx

  def forward(self, src, tgt):
    src_seq_len, batch_size = src.shape
    tgt_seq_len, _ = tgt.shape

    src_positions = (
        torch.arange(0, src_seq_len).unsqueeze(1).expand(src_seq_len, batch_size)
        .to(self.device)
    )

    tgt_positions = (
        torch.arange(0, tgt_seq_len).unsqueeze(1).expand(tgt_seq_len, batch_size)
        .to(self.device)
    )

    emb_src = self.dropout(
        self.src_word_embedding(src) + self.src_position_embedding(src_positions)
    )

    emb_tgt = self.dropout(
        self.tgt_word_embedding(tgt) + self.tgt_position_embedding(tgt_positions)
    )

    src_padding_mask = self.create_padding_mask(src, self.src_pad_idx)
    tgt_padding_mask = self.create_padding_mask(tgt, self.tgt_pad_idx)
    tgt_mask = self.transformer.generate_square_subsequent_mask(tgt_seq_len).to(self.device)

    out = self.transformer(
        emb_src,
        emb_tgt,
        tgt_mask=tgt_mask,
        src_key_padding_mask=src_padding_mask,
        tgt_key_padding_mask=tgt_padding_mask
    )

    return self.fc_out(out)

### Utility functions

In [ ]:
def batch_translate_tensor(model, batch_data, max_length=MAX_SEQ_LEN, detokenize=True):
  data_len, seq_len = batch_data.shape

  model.eval()

  src = batch_data.T.to(device)
  tgt = torch.full(
      (1, data_len),
      dataset.tgt_tok2idx[START_TOKEN],
      dtype=torch.long,
      device=device
  )

  finished = torch.zeros(data_len, dtype=torch.bool, device=device)

  for _ in range(max_length-1):
    with torch.no_grad():
      output = model(src, tgt)

    next_token_logits = output[-1, :, :]
    next_tokens = next_token_logits.argmax(dim=1)

    next_tokens = next_tokens.masked_fill(finished, dataset.src_tok2idx[END_TOKEN])
    tgt = torch.cat([tgt, next_tokens.unsqueeze(0)], dim=0)

    finished |= (next_tokens == dataset.src_tok2idx[END_TOKEN])
    if finished.all():
      break

  if not detokenize:
    return tgt.T

  translations = []
  tgt_np = tgt.detach().cpu().numpy().T

  for seq in tgt_np:
    words = []
    for idx in seq[1:]:
      if idx == dataset.tgt_tok2idx[END_TOKEN]:
        break
      words.append(dataset.tgt_idx2tok[idx])
    translations.append(" ".join(words))

  return translations

def en_token_rejoin(tokens):
  sentence = ""
  for token in tokens:
    if token[0] in ["'", ".", ",", "!", "?", ":", ";"]:
      sentence = sentence.rstrip() + token
      continue

    sentence+= token + " "

  return sentence

def translate_sentence(model, sentence, max_length=MAX_SENT_LEN):
  tokens = [token.text.lower() for token in spacy_fr(sentence)]

  tokens.insert(0, START_TOKEN)
  tokens.append(END_TOKEN)

  text_to_indices = [ dataset.src_tok2idx.get(token, dataset.src_tok2idx[UNKN_TOKEN]) for token in tokens]
  sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

  outputs = [dataset.tgt_tok2idx[START_TOKEN]]
  for i in range(max_length):
      tgt_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

      with torch.no_grad():
          output = model(sentence_tensor, tgt_tensor)

      best_guess = output.argmax(2)[-1, :].item()
      if best_guess == dataset.tgt_tok2idx[END_TOKEN]:
          break

      outputs.append(best_guess)

  translated_sentence = [dataset.tgt_idx2tok[idx] for idx in outputs]

  return en_token_rejoin(translated_sentence[1:])

def bleu_score(model, test_iter, max_count=None, verbose=True):
  tgts = []
  pred_tgts = []
  cnt = 0

  for (x, y) in test_iter:
    src = x.to(device)
    tgt = y[:, :-1]

    pred = batch_translate_tensor(model, src)
    pred_tgts.extend(pred)
    for sent_tok in tgt:
      tgts.append([" ".join([ val for tok in sent_tok if (val:=dataset.tgt_idx2tok[tok.item()]) not in [START_TOKEN, END_TOKEN, PAD_TOKEN]])])

    cnt += src.shape[0]
    if max_count and cnt >= max_count:
      break

  bleu = BLEUScore().to(device)
  score = bleu(pred_tgts, tgts)

  if(verbose):
    print(f"Total tested datapoints: {cnt}")
    print(f"BLEU score: {score * 100:.2f}%")

  return score.item()

### Misc definitions

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

spacy_fr = spacy.load("fr_core_news_sm")
spacy_en = spacy.load("en_core_web_sm")

tokenize_fr = lambda text: [ text for tok in spacy_fr.tokenizer(text) if (text:=tok.text.strip()) ]
tokenize_en = lambda text: [ text for tok in spacy_en.tokenizer(text) if (text:=tok.text.strip()) ]

### Loading dataset and vocabularies

In [ ]:
batch_size = 512
train_test_split = [0.9, 0.1]

dataset = LangDataset(FR_PATH, EN_PATH, tokenize_fr, tokenize_en, max_lines=MAX_LINE_COUNT)

train_data, test_data = torch.utils.data.random_split(dataset, train_test_split)
train_iter = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_iter = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Reading source and target files...
Reading source files! Read 2007723 lines. Keeping 200000 lines.
Preprocessing dataset...
Preprocessing completed! Dropped 1317/200000 lines!
Creating vocabularies...
Created vocabularies! Source vocab size: 52869 and target vocab size: 37116
Tensorifying dataset...
Dataset tensorified!


### Training

In [ ]:
def train(train_iter, val_iter, dataset, device, config):
  embedding_size = config["embedding_size"]
  num_heads = config["num_heads"]
  num_encoder_layers = config["num_encoder_layers"]
  num_decoder_layers = config["num_decoder_layers"]
  dropout = config["dropout"]
  learning_rate = config["learning_rate"]
  num_epochs = config["num_epochs"]

  log_per = 1
  dim_feedforward = 4 * embedding_size

  src_pad_idx = dataset.src_tok2idx[PAD_TOKEN]
  tgt_pad_idx = dataset.tgt_tok2idx[PAD_TOKEN]
  tgt_start_idx = dataset.tgt_tok2idx[START_TOKEN]
  tgt_end_idx = dataset.tgt_tok2idx[END_TOKEN]

  model = Transformer(
    embedding_size,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    dim_feedforward,
    dropout,
    dataset.src_vocab_size,
    dataset.tgt_vocab_size,
    src_pad_idx,
    tgt_pad_idx,
    device
  ).to(device)

  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  criterion = nn.CrossEntropyLoss(ignore_index=tgt_pad_idx)

  loss_history = []
  acc_history = []

  for epoch in range(num_epochs):
    batch_loss = []
    batch_acc = []

    model.train()
    for src, target in train_iter:
      src = src.to(device).T
      target = target.to(device).T

      target_input  = target[:-1, :]
      target_output = target[1:, :]

      output = model(src, target_input)
      pred = output.argmax(2)

      mask = (
        (target_output != tgt_end_idx) &
        (target_output != tgt_pad_idx)
      )

      correct  = (pred == target_output) & mask
      accuracy = correct.sum() / mask.sum()
      batch_acc.append(accuracy.item())

      output_flat = output.reshape(-1, output.shape[2])
      target_flat = target_output.reshape(-1)

      optimizer.zero_grad()
      loss = criterion(output_flat, target_flat)
      loss.backward()
      batch_loss.append(loss.item())

      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
      optimizer.step()

    avg_acc  = sum(batch_acc) / len(batch_acc)
    avg_loss = sum(batch_loss) / len(batch_loss)
    acc_history.append(avg_acc)
    loss_history.append(avg_loss)

    torch.save({
      "loss_history": loss_history,
      "acc_history": acc_history,
      "epoch": epoch,
      "datapoints": MAX_LINE_COUNT,
      "model_state_dict": model.state_dict(),
      "optimizer_state_dict": optimizer.state_dict(),
      }, TRAIN_STATS_PATH)

    if epoch % log_per == 0:
      print(f"Epoch {epoch + 1}/{num_epochs} - loss: {avg_loss:.4f} - train acc: {avg_acc * 100:.2f}%")

  test_bleu = bleu_score(model, test_iter, verbose=False)
  print(f"Epoch {epoch + 1}/{num_epochs} - loss: {avg_loss:.4f} - train acc: {avg_acc * 100:.2f}%")
  print(f"Test BLEU: {test_bleu * 100:.2f}%")

  torch.save({
  "loss_history": loss_history,
  "acc_history": acc_history,
  "epoch": epoch,
  "datapoints": MAX_LINE_COUNT,
  "model_state_dict": model.state_dict(),
  "optimizer_state_dict": optimizer.state_dict(),
  "test_bleu": test_bleu,
  }, TRAIN_STATS_PATH)

  return test_bleu, model, loss_history, acc_history

In [ ]:
config = {
  "embedding_size": 512,
  "num_heads": 4,
  "num_encoder_layers": 2,
  "num_decoder_layers": 3,
  "dropout": 0.1,
  "learning_rate": 3e-4,
  "num_epochs":100
}

test_bleu, model, loss_hist, acc_hist = train(
    train_iter, test_iter, dataset, device, config
  )

Epoch 1/100 - loss: 5.0561 - train acc: 22.74%
Epoch 2/100 - loss: 3.8038 - train acc: 35.29%
Epoch 3/100 - loss: 3.2336 - train acc: 41.89%
Epoch 4/100 - loss: 2.8721 - train acc: 46.13%
Epoch 5/100 - loss: 2.6304 - train acc: 48.96%
Epoch 6/100 - loss: 2.4537 - train acc: 51.01%
Epoch 7/100 - loss: 2.3153 - train acc: 52.68%
Epoch 8/100 - loss: 2.2020 - train acc: 54.05%
Epoch 9/100 - loss: 2.1083 - train acc: 55.20%
Epoch 10/100 - loss: 2.0270 - train acc: 56.26%
Epoch 11/100 - loss: 1.9581 - train acc: 57.13%
Epoch 12/100 - loss: 1.8959 - train acc: 57.98%
Epoch 13/100 - loss: 1.8404 - train acc: 58.73%
Epoch 14/100 - loss: 1.7902 - train acc: 59.43%
Epoch 15/100 - loss: 1.7454 - train acc: 60.06%
Epoch 16/100 - loss: 1.7037 - train acc: 60.65%
Epoch 17/100 - loss: 1.6647 - train acc: 61.23%
Epoch 18/100 - loss: 1.6304 - train acc: 61.72%
Epoch 19/100 - loss: 1.5978 - train acc: 62.18%
Epoch 20/100 - loss: 1.5659 - train acc: 62.66%
Epoch 21/100 - loss: 1.5372 - train acc: 63.10%
E

In [ ]:
torch.save({
  "src_idx2tok": dataset.src_idx2tok,
  "src_tok2idx": dataset.src_tok2idx,
  "tgt_idx2tok": dataset.tgt_idx2tok,
  "tgt_tok2idx": dataset.tgt_tok2idx,

  }, os.path.join(ROOT_PATH, "vocab.pth"))

In [ ]:
input_sent = "je suis"
actual_translation = "I am in the process of"
model.eval()
prediction = translate_sentence(model, input_sent)
print(f"Input sentence: {input_sent}")
print(f"Translated sentence: {prediction}")
print(f"Actual translation: {actual_translation}")